In [19]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
sns.set_style("darkgrid")
warnings.filterwarnings('ignore')

In [20]:
folder = ''
temp1 = pd.read_csv(folder+'Task2_step3.csv')
print(temp1.columns)
temp1.head()

Index(['participant_ID', 'treatment', 'part', 'trial_no', 'setup_cost_red',
       'setup_cost_blue', 'red_prior_prob', 'true_guilty_suspect',
       'suspect_accused', 'choice_1',
       ...
       'timing_choice_55', 'timing_choice_56', 'timing_choice_57',
       'timing_choice_58', 'timing_choice_59', 'timing_choice_60',
       'timing_choice_61', 'timing_choice_62', 'correct_suspect_accused',
       'evidence_round'],
      dtype='object', length=193)


,participant_ID,treatment,part,trial_no,setup_cost_red,setup_cost_blue,red_prior_prob,true_guilty_suspect,suspect_accused,choice_1,...,timing_choice_55,timing_choice_56,timing_choice_57,timing_choice_58,timing_choice_59,timing_choice_60,timing_choice_61,timing_choice_62,correct_suspect_accused,evidence_round
0,1,0,2,1,80.0,80.0,0.7,1,0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,1,0,2,2,20.0,20.0,0.7,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
2,1,0,2,3,40.0,40.0,0.5,1,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
3,1,0,2,4,40.0,40.0,0.3,1,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4,1,0,2,5,80.0,80.0,0.1,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0


In [21]:
data = []
for key,value in temp1.iterrows():
    info_b = value[:7].to_list()
    info_e = value[7:9].to_list()
    rounds = value[9:69].to_list()
    rounds = [x for x in rounds if x == 1.0 or x == 0.0]
    suspect = []
    post = value[6]
    
    e_f = 0
    e_s = -1
    i = 0
    for i in range(len(rounds)):
        suspect.append(rounds[i])
        pg = 0.75 if suspect[-1] == 0 else 1
        pi = 1 if suspect[-1] == 0 else 0.75
        temp = [i+1, 0, suspect[-1], suspect[:-1].count(0.0), 
               suspect[:-1].count(1.0), e_f, e_s]
        data.append(info_b+temp+[post]+info_e+[temp1['timing_choice_'+str(i+1)][key]])
        post = post*pg/(post*pg + pi*(1-post))
        
        e_f = int(value[69+i]) if e_f != 1 else 1
        if e_f == 1: 
            e_s = value[8]
        else: e_f = 1 if e_f == 1 and i != 0 else 0
        post = value[7] if e_f == 1 else post
    suspect.append(value[7])
    temp = [i+2, 1, suspect[-1], suspect[:-1].count(0.0), suspect[:-1].count(1.0), e_f, e_s]
    data.append(info_b+temp+[post]+info_e+[temp1['timing_choice_'+str(i+2)][key]])

In [22]:
cols = ['participant_ID', 'treatment', 'part', 'trial_no', 'setup_cost_red', 'setup_cost_blue',
       'red_prior_prob', 'current_rounds', 'action_type', 'suspect', 
        'count_red_samples', 'count_blue_samples', 'evidence_found',
       'evidence_suspect', 'posterior', 'true_guilty_suspect', 'suspect_accused',
       'time']
df2 = pd.DataFrame(data,columns=cols)
df2.head()

,participant_ID,treatment,part,trial_no,setup_cost_red,setup_cost_blue,red_prior_prob,current_rounds,action_type,suspect,count_red_samples,count_blue_samples,evidence_found,evidence_suspect,posterior,true_guilty_suspect,suspect_accused,time
0,1.0,0.0,2.0,1.0,80.0,80.0,0.7,1,0,1.0,0,0,0,-1.0,0.700000,1.0,0.0,26776.0609
1,1.0,0.0,2.0,1.0,80.0,80.0,0.7,2,1,1.0,0,1,0,-1.0,0.756757,1.0,0.0,28743.0029
2,1.0,0.0,2.0,2.0,20.0,20.0,0.7,1,0,0.0,0,0,0,-1.0,0.700000,0.0,0.0,4094.1187
3,1.0,0.0,2.0,2.0,20.0,20.0,0.7,2,1,0.0,1,0,1,0.0,0.000000,0.0,0.0,5877.7017
4,1.0,0.0,2.0,3.0,40.0,40.0,0.5,1,0,1.0,0,0,0,-1.0,0.500000,1.0,1.0,1716.9070


In [23]:
df2 = df2.fillna(0)
df2.to_csv('Task2_step4.csv', index=False)

In [24]:
0.5*1/(0.5*1 + 0.75*0.5)

0.5714285714285714